In [3]:
# In a Jupyter Notebook cell
%load_ext autoreload
%autoreload 2

from mirage.index import WordCountingChunkingAlgorithm, FolderRawStorage, SQLiteChunkStorage, RAMChunkStorage
from mirage.embedders import HuggingFaceEmbedder, TextNormalizer
import os
import pickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


c:\Users\User\Desktop\Диплом\MIRAGE\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import logging
logging.basicConfig(level=logging.ERROR)


documents = FolderRawStorage('data')
chunks = RAMChunkStorage()
algorithm = WordCountingChunkingAlgorithm(documents, chunks, words_amount=100)
algorithm.execute(visualize=True)

13

In [5]:
index = chunks.get_indexes()[0]
chunks[index]

'Приказ Росавиации от 28.12.2022 N 970-П\n"Об утверждении Норм летной годности\nгражданских легких самолетов НЛГ 23"\nДокумент предоставлен КонсультантПлюс\nwww.consultant.ru\nДата сохранения: 09.01.2024\nПриказ Росавиации от 28.12.2022 N 970-П\nДокумент предоставленКонсультантПлюс\n"Об утверждении Норм летной годности гражданских\nДата сохранения: 09.01.2024\nлегких самолетов НЛГ 23"\nМИНИСТЕРСТВО ТРАНСПОРТА РОССИЙСКОЙ ФЕДЕРАЦИИ\nФЕДЕРАЛЬНОЕ АГЕНТСТВО ВОЗДУШНОГО ТРАНСПОРТА\nПРИКАЗ\nот 28 декабря 2022 г. N 970-П\nОБ УТВЕРЖДЕНИИ НОРМ\nЛЕТНОЙ ГОДНОСТИ ГРАЖДАНСКИХ ЛЕГКИХ САМОЛЕТОВ НЛГ 23\nВ соответствии с пунктом 2 статьи 35 Воздушного кодекса Российской Федерации и подпунктами\n5.14.4 и 9.9 Положения о Федеральном агентстве воздушного транспорта, утвержденного постановлением\nПравительства Российской Федерации от 30 июля 2004 г. N 396, приказываю:\n1. Утвердить прилагаемыеНормы летной годности гражданских легких самолетов НЛГ 23.\n2. Настоящий приказ вступает в силу'

In [6]:
from mirage.embedders.BowEmbedder import BowEmbedder

emb = HuggingFaceEmbedder(normalizer = True)
emb.fit(chunks)

c:\Users\User\Desktop\Диплом\MIRAGE\env\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\.cache\huggingface\hub\models--sentence-transformers--distiluse-base-multilingual-cased-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [7]:
emb.get_dimensionality()

512

In [8]:
from mirage.index.vector_index.ram_vector_index import L2RAMVectorIndex

vector_index = L2RAMVectorIndex(dimensionality = emb.get_dimensionality())
emb.convert_chunks_to_vector_index(chunks, vector_index, visualize=True)

Converting ChunkStorage to VectorIndex


100%|██████████| 2660/2660 [04:01<00:00, 11.02it/s]


In [16]:
query = 'Какой пилот самолёта?'
answer = vector_index.query(
    emb.embed(
        text=query 
    ), top_k=5
)
print([element.distance for element in answer])
[chunks[element.chunk_storage_key] for element in answer]

[np.float32(-0.090230174), np.float32(-0.08811018), np.float32(-0.08783565), np.float32(-0.08416579), np.float32(-0.07956384)]


['и\nработе при наличии состояний слезных органов и слезоотводящих путей, связанных с нарушением\nслезоотведения.\nК данным заболеваниям относятся: сужение, неправильное положение слезных точек, стриктура\nслезно-носовых канальцев, воспаление слезного мешка, слезной железы, травматические повреждения и\nдругие заболевания, вызывающие слезотечение.\n282. Состояния слезных органов и слезоотводящих путей без нарушения слезоотведения,\nвосстановление функции слезоотведения, а также отсутствие нижней слезной точки при нормальной\nфункции верхней слезной точки и канальцев (оцениваются канальцевая и носовая пробы) не являются\nдисквалифицирующим фактором при определении годности к работе.\n283. После перенесенного острого заболевания - конъюнктивита, склерита, кератита, переднего\nувеита, легких повреждениях глазного яблока - заключение о допуске к летной работе выносится после\nвыздоровления и прекращения лечения, так как некоторые глазные препараты',
 'вопрос о годности рассматривается посл

In [19]:
from mirage.inference.prompters.APILLM import LLM
llm = LLM()
llm.do_request(query=query,
               chanks_storage=chunks,
               indexes=[element.chunk_storage_key for element in answer],
               promt='')

Ответь на вопрос:Какой пилот самолёта?
Зная эту информацию:
------
и
работе при наличии состояний слезных органов и слезоотводящих путей, связанных с нарушением
слезоотведения.
К данным заболеваниям относятся: сужение, неправильное положение слезных точек, стриктура
слезно-носовых канальцев, воспаление слезного мешка, слезной железы, травматические повреждения и
другие заболевания, вызывающие слезотечение.
282. Состояния слезных органов и слезоотводящих путей без нарушения слезоотведения,
восстановление функции слезоотведения, а также отсутствие нижней слезной точки при нормальной
функции верхней слезной точки и канальцев (оцениваются канальцевая и носовая пробы) не являются
дисквалифицирующим фактором при определении годности к работе.
283. После перенесенного острого заболевания - конъюнктивита, склерита, кератита, переднего
увеита, легких повреждениях глазного яблока - заключение о допуске к летной работе выносится после
выздоровления и прекращения лечения, так как некоторые глазные